### Vectorization of functions

* functions are typically not "vectorized" in Julia
* you have to explicitly use the "dot-notation"

### Broadcasting different shapes

* dimensions of length 1 will be extended
* can be used e.g. to subtract columns means
* or for outer products
* left-hand side arrays can be written to using .=

### Broadcast loop fusion

In [6]:
f(x,y,z) = sum(exp.(x .- y) .* z .- x) 

f (generic function with 1 method)

In [7]:
x,y,z = rand(1000), rand(1000), rand(1000);

In [9]:
@time f(x,y,z)

  0.000075 seconds (6 allocations: 8.109 KiB)


50.32626161855468

### Exercises

1. Try broadcasting some functions